<a href="https://colab.research.google.com/github/PARTHIBAN-007/LLM-FineTuning/blob/main/Gemma_3_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.12 requires tyro, which is not installed.
unsloth-zoo 2025.3.12 requires protobuf<4.0.0, bu

In [2]:
from unsloth import FastModel
import torch


model , tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers = False,
    finetune_language_layers = True,
    finetune_attention_modules = True,
    finetune_mlp_modules = True,
    r= 4,
    lora_alpha = 4,
    lora_dropout = 0,
    bias = "none",
    random_state = 42,
)

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer ,
    chat_template = "gemma-3"
)

In [5]:
from datasets import load_dataset

dataset = load_dataset("mlabonne/FineTome-100k",split="train")


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [6]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [7]:
dataset[250]

{'conversations': [{'content': 'Write Python code to solve the task:\nA positive integer is called a palindrome if its representation in the \n\ndecimal system is the same when read from left to right and from right \n\nto left. For a given positive integer K of not more than 5 digits, \n\nwrite the value of the smallest palindrome larger than K to output. \n\nNumbers are always displayed without leading zeros.\n\n\nInput\n\n\n\nThe first line contains integer t, the number of test cases. \n\nIntegers K are given in the next t lines.\n\n\n\n\nOutput\n\n\nFor each K, output the smallest palindrome larger than K.\n\n\n\nExample\n\nInput:\n1\n808\n\nOutput:\n818',
   'role': 'user'},
  {'content': "Step 1:  The solution requires a loop to iterate through each test case, and for each test case, find the smallest palindrome larger than the input.\nStep 2:  To find the smallest palindrome larger than the input, it should keep increasing the input number by one and checking if the new number 

In [10]:
def apply_chat_template(example):
  texts = tokenizer.apply_chat_template(example["conversations"])
  return {"text":texts}
pass
dataset = dataset.map(apply_chat_template,batched =True)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [11]:
dataset[250]["text"]

"<bos><start_of_turn>user\nWrite Python code to solve the task:\nA positive integer is called a palindrome if its representation in the \n\ndecimal system is the same when read from left to right and from right \n\nto left. For a given positive integer K of not more than 5 digits, \n\nwrite the value of the smallest palindrome larger than K to output. \n\nNumbers are always displayed without leading zeros.\n\n\nInput\n\n\n\nThe first line contains integer t, the number of test cases. \n\nIntegers K are given in the next t lines.\n\n\n\n\nOutput\n\n\nFor each K, output the smallest palindrome larger than K.\n\n\n\nExample\n\nInput:\n1\n808\n\nOutput:\n818<end_of_turn>\n<start_of_turn>model\nStep 1:  The solution requires a loop to iterate through each test case, and for each test case, find the smallest palindrome larger than the input.\nStep 2:  To find the smallest palindrome larger than the input, it should keep increasing the input number by one and checking if the new number is a p

In [16]:
from warnings import WarningMessage
from trl import SFTTrainer  , SFTConfig

training_args = SFTConfig(
    dataset_text_field = "text",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-4,
    warmup_steps = 5,
    max_steps = 30,
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = training_args,
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [19]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [20]:
tokenizer.decode(trainer.train_dataset[250]["input_ids"])

"<bos><start_of_turn>user\nWrite Python code to solve the task:\nA positive integer is called a palindrome if its representation in the \n\ndecimal system is the same when read from left to right and from right \n\nto left. For a given positive integer K of not more than 5 digits, \n\nwrite the value of the smallest palindrome larger than K to output. \n\nNumbers are always displayed without leading zeros.\n\n\nInput\n\n\n\nThe first line contains integer t, the number of test cases. \n\nIntegers K are given in the next t lines.\n\n\n\n\nOutput\n\n\nFor each K, output the smallest palindrome larger than K.\n\n\n\nExample\n\nInput:\n1\n808\n\nOutput:\n818<end_of_turn>\n<start_of_turn>model\nStep 1:  The solution requires a loop to iterate through each test case, and for each test case, find the smallest palindrome larger than the input.\nStep 2:  To find the smallest palindrome larger than the input, it should keep increasing the input number by one and checking if the new number is a p

In [23]:
tokenizer.decode([tokenizer.pad_token_id if x==-100 else x for x in trainer.train_dataset[250]["labels"]]).replace(tokenizer.pad_token, " ")


"                                                                                                                                                        Step 1:  The solution requires a loop to iterate through each test case, and for each test case, find the smallest palindrome larger than the input.\nStep 2:  To find the smallest palindrome larger than the input, it should keep increasing the input number by one and checking if the new number is a palindrome.\nStep 3:  This process continues until the number is a palindrome, and then the number is printed as the smallest palindrome larger than the input.\n```python\n# Step 4:  Loop through each test case\nfor t in range(int(input())):\n    # Step 5:  Read the input number\n    \n    n = int(input()) + 1\n    # Step 6:  Increment the number and check if it's a palindrome until a palindrome is found\n    \n    while (str(n) != str(n)[::-1]):\n        n += 1\n    # Step 7:  Print the smallest palindrome larger than the input\n    \n    p

In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 7,450,624/4,000,000,000 (0.19% trained)
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.237700
2,1.636400
3,1.766300
4,1.420700
5,1.235700
6,1.806600
7,1.010100
8,1.896600
9,1.464700
10,1.309700


In [27]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer ,
    chat_template = "gemma-3",
)
messages = [{
    "role":"user",
    "content":[{
        "type":"text",
        "text": "continue in the sequence : 1,1,2,3,5,8"
    }]
}]

text = tokenizer.apply_chat_template(
    messages ,
    add_generation_prompt = True,
)

outputs = model.generate(
   **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 300,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)


tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\ncontinue in the sequence : 1,1,2,3,5,8<end_of_turn>\n<start_of_turn>model\n1, 1, 2, 3, 5, 8, **13, 21, 34**\n\nThis is the Fibonacci sequence. Each number is the sum of the two preceding ones. (1+1=2, 1+2=3, 2+3=5, 3+5=8, 5+8=13, 8+13=21, 13+21=34, etc.)\n<end_of_turn>']

In [26]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64,
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a fascinating phenomenon that boils down to a combination of physics and light. Here's the explanation:

**1. Sunlight and its Colors:**

* Sunlight, which appears white to us, is actually made up of *all* the


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
model.save_pretrained("gemma-3")
tokenizer.save_pretrained("gemma-3")
model.push_to_hub("Parthiban007/gemma-3")
tokenizer.push_to_hub("Parthiban007/gemma-3")

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/29.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Parthiban007/gemma-3


  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]